# Lab 13: Memory Forensics AI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab13_memory_forensics.ipynb)

Use AI to analyze memory dumps and detect advanced threats.

## Learning Objectives
- Volatility3 integration for memory analysis
- Process injection detection
- Credential dumping identification
- LLM-powered artifact interpretation

In [ ]:
# Install dependencies
!pip install anthropic pandas -q

In [ ]:
import json
import pandas as pd
from typing import List, Dict
from dataclasses import dataclass

## 1. Memory Analysis Concepts

Key artifacts to analyze:
- **Processes**: Running executables
- **DLLs**: Loaded libraries per process
- **Network**: Active connections
- **Registry**: In-memory registry hives
- **Handles**: Open file/registry handles

In [ ]:
# Simulated Volatility3 output (processes)
SAMPLE_PROCESSES = [
    {"pid": 4, "ppid": 0, "name": "System", "path": "", "cmdline": ""},
    {"pid": 456, "ppid": 4, "name": "smss.exe", "path": "C:\\Windows\\System32\\smss.exe", "cmdline": ""},
    {"pid": 568, "ppid": 456, "name": "csrss.exe", "path": "C:\\Windows\\System32\\csrss.exe", "cmdline": ""},
    {"pid": 1234, "ppid": 568, "name": "cmd.exe", "path": "C:\\Windows\\System32\\cmd.exe", "cmdline": "cmd.exe /c whoami"},
    {"pid": 5678, "ppid": 1234, "name": "powershell.exe", "path": "C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe", 
     "cmdline": "powershell -enc JABjAGwAaQBlAG4AdAA="},
    {"pid": 9999, "ppid": 5678, "name": "svchost.exe", "path": "C:\\Users\\Public\\svchost.exe", "cmdline": ""},
]

df = pd.DataFrame(SAMPLE_PROCESSES)
print("Process List:")
df

## 2. Detecting Suspicious Processes

In [ ]:
@dataclass
class ProcessAnomaly:
    pid: int
    name: str
    anomaly_type: str
    description: str
    severity: str

def detect_process_anomalies(processes: List[Dict]) -> List[ProcessAnomaly]:
    """Detect suspicious process patterns."""
    anomalies = []
    
    # Known system paths
    SYSTEM32 = "C:\\Windows\\System32"
    
    for proc in processes:
        # Check for masquerading (wrong path)
        if proc['name'].lower() == 'svchost.exe' and SYSTEM32 not in proc['path']:
            anomalies.append(ProcessAnomaly(
                pid=proc['pid'],
                name=proc['name'],
                anomaly_type="masquerading",
                description=f"svchost.exe running from non-standard path: {proc['path']}",
                severity="CRITICAL"
            ))
        
        # Check for encoded PowerShell
        if 'powershell' in proc['name'].lower() and '-enc' in proc.get('cmdline', ''):
            anomalies.append(ProcessAnomaly(
                pid=proc['pid'],
                name=proc['name'],
                anomaly_type="encoded_command",
                description="PowerShell with encoded command",
                severity="HIGH"
            ))
    
    return anomalies

# Detect anomalies
anomalies = detect_process_anomalies(SAMPLE_PROCESSES)
for a in anomalies:
    print(f"[{a.severity}] PID {a.pid}: {a.anomaly_type} - {a.description}")

## 3. DLL Injection Detection

In [ ]:
# Simulated DLL list
SAMPLE_DLLS = [
    {"pid": 1234, "dll": "kernel32.dll", "path": "C:\\Windows\\System32\\kernel32.dll"},
    {"pid": 1234, "dll": "ntdll.dll", "path": "C:\\Windows\\System32\\ntdll.dll"},
    {"pid": 1234, "dll": "evil.dll", "path": "C:\\Users\\Public\\evil.dll"},  # Suspicious!
    {"pid": 5678, "dll": "amsi.dll", "path": ""},  # Unlinked - potentially hollowed
]

def detect_dll_anomalies(dlls: List[Dict]) -> List[Dict]:
    """Detect suspicious DLL patterns."""
    suspicious = []
    
    for dll in dlls:
        # Check for DLLs from user directories
        if 'Users' in dll.get('path', ''):
            suspicious.append({
                "pid": dll['pid'],
                "dll": dll['dll'],
                "type": "user_path_dll",
                "description": f"DLL loaded from user directory: {dll['path']}"
            })
        
        # Check for unlinked DLLs (potential hollowing)
        if not dll.get('path'):
            suspicious.append({
                "pid": dll['pid'],
                "dll": dll['dll'],
                "type": "unlinked_dll",
                "description": "DLL with no path - possible hollowing"
            })
    
    return suspicious

dll_anomalies = detect_dll_anomalies(SAMPLE_DLLS)
for a in dll_anomalies:
    print(f"PID {a['pid']}: {a['type']} - {a['description']}")

## 4. LLM-Powered Analysis

In [ ]:
from anthropic import Anthropic

def analyze_memory_artifacts(processes: List[Dict], dlls: List[Dict]) -> str:
    """Use LLM to analyze memory artifacts."""
    
    client = Anthropic()
    
    prompt = f"""
    You are a memory forensics expert. Analyze these artifacts from a Windows memory dump.
    
    PROCESSES:
    {json.dumps(processes, indent=2)}
    
    LOADED DLLs:
    {json.dumps(dlls, indent=2)}
    
    Provide:
    1. Summary of findings
    2. Identified threats (with MITRE ATT&CK techniques)
    3. Indicators of Compromise
    4. Recommended next steps
    """
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2000,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.content[0].text

# Uncomment to run with API key
# analysis = analyze_memory_artifacts(SAMPLE_PROCESSES, SAMPLE_DLLS)
# print(analysis)

## 5. Credential Detection

In [ ]:
# Patterns indicating credential access
CREDENTIAL_INDICATORS = [
    {"pattern": "lsass.exe", "type": "process", "technique": "T1003.001"},
    {"pattern": "sekurlsa", "type": "string", "technique": "T1003.001"},
    {"pattern": "mimikatz", "type": "string", "technique": "T1003"},
    {"pattern": "procdump", "type": "cmdline", "technique": "T1003.001"},
]

def check_credential_access(processes: List[Dict]) -> List[Dict]:
    """Check for credential dumping indicators."""
    findings = []
    
    for proc in processes:
        cmdline = proc.get('cmdline', '').lower()
        name = proc.get('name', '').lower()
        
        # Check for lsass access
        if 'lsass' in cmdline:
            findings.append({
                "pid": proc['pid'],
                "indicator": "LSASS access in command line",
                "technique": "T1003.001",
                "severity": "CRITICAL"
            })
        
        # Check for known tools
        if 'mimikatz' in name or 'procdump' in cmdline:
            findings.append({
                "pid": proc['pid'],
                "indicator": "Credential dumping tool detected",
                "technique": "T1003",
                "severity": "CRITICAL"
            })
    
    return findings

cred_findings = check_credential_access(SAMPLE_PROCESSES)
print(f"Credential access findings: {len(cred_findings)}")

## Key Takeaways

1. **Process Analysis**: Check paths, parent-child relationships, command lines
2. **DLL Injection**: Look for unusual paths, unlinked modules
3. **Credential Access**: Monitor LSASS access patterns
4. **LLM Enhancement**: Use AI to correlate and explain findings

## Next Steps
- **Lab 14**: C2 Traffic Analysis
- **Lab 15**: Lateral Movement Detection